# Import data from csv files

In [1]:
import pandas as pd
df = pd.read_csv('DBC/Field Bus1.BrowerRadiant2East.ZN2-T.csv')

In [67]:
# transfer the unit from F to C
df['Celcius'] = (df['/FS4/NAE00806603BECA/3001087/NAE00806603BECA/Field Bus1.BrowerRadiant2East.ZN2-T']-32)*5/9

In [92]:
df

,datetime,/FS4/NAE00806603BECA/3001087/NAE00806603BECA/Field Bus1.BrowerRadiant2East.ZN2-T,Celcius
0,2020-12-01 10:05:00,70.299370,21.277428
1,2020-12-01 10:10:00,70.299370,21.277428
2,2020-12-01 10:15:00,70.299370,21.277428
3,2020-12-01 10:20:00,70.299370,21.277428
4,2020-12-01 10:25:00,70.502136,21.390076
...,...,...,...
34556,2021-03-31 10:45:00,67.608704,19.782613
34557,2021-03-31 10:50:00,67.608704,19.782613
34558,2021-03-31 10:55:00,67.820107,19.900059
34559,2021-03-31 11:00:00,67.820107,19.900059


In [130]:
df.columns[0]

'datetime'

In [129]:
df['weekdays'] = pd.to_datetime(df['datetime']).dt.dayofweek
df[(df['weekdays'] == 6)]

,datetime,/FS4/NAE00806603BECA/3001087/NAE00806603BECA/Field Bus1.BrowerRadiant2East.ZN2-T,Celcius,weekdays


In [68]:
ts_diff = pd.to_datetime(df['datetime'][1]) - pd.to_datetime(df['datetime'][0])
ts_sec= ts_diff.seconds

In [69]:
# calculate the interval between two time

In [112]:
df[(df['Celcius'] < 15)]

,datetime,/FS4/NAE00806603BECA/3001087/NAE00806603BECA/Field Bus1.BrowerRadiant2East.ZN2-T,Celcius
8864,2021-01-01 04:45:00,58.866699,14.925944
8865,2021-01-01 04:50:00,58.866699,14.925944
8866,2021-01-01 04:55:00,58.866699,14.925944
8867,2021-01-01 05:00:00,58.866699,14.925944
8868,2021-01-01 05:05:00,58.866699,14.925944
8869,2021-01-01 05:10:00,58.866699,14.925944
8870,2021-01-01 05:15:00,58.866699,14.925944
8871,2021-01-01 05:20:00,58.866699,14.925944
8872,2021-01-01 05:25:00,58.866699,14.925944
8873,2021-01-01 05:30:00,58.866699,14.925944


In [108]:
df[(df['Celcius'] >23)]

,datetime,/FS4/NAE00806603BECA/3001087/NAE00806603BECA/Field Bus1.BrowerRadiant2East.ZN2-T,Celcius
615,2020-12-03 13:20:00,73.410248,23.005693
616,2020-12-03 13:25:00,73.410248,23.005693
617,2020-12-03 13:30:00,73.410248,23.005693
618,2020-12-03 13:35:00,73.410248,23.005693
619,2020-12-03 13:40:00,73.410248,23.005693
...,...,...,...
25171,2021-02-26 19:40:00,73.434921,23.019401
25172,2021-02-26 19:45:00,73.434921,23.019401
25173,2021-02-26 19:50:00,73.434921,23.019401
25174,2021-02-26 19:55:00,73.434921,23.019401


In [110]:
df[(df['Celcius'] < 22) | (df['Celcius'] > 27)]

,datetime,/FS4/NAE00806603BECA/3001087/NAE00806603BECA/Field Bus1.BrowerRadiant2East.ZN2-T,Celcius
0,2020-12-01 10:05:00,70.299370,21.277428
1,2020-12-01 10:10:00,70.299370,21.277428
2,2020-12-01 10:15:00,70.299370,21.277428
3,2020-12-01 10:20:00,70.299370,21.277428
4,2020-12-01 10:25:00,70.502136,21.390076
...,...,...,...
34556,2021-03-31 10:45:00,67.608704,19.782613
34557,2021-03-31 10:50:00,67.608704,19.782613
34558,2021-03-31 10:55:00,67.820107,19.900059
34559,2021-03-31 11:00:00,67.820107,19.900059


In [44]:
comfort_sec = len(comfort_df) * ts_sec
total_sec = len(df) * ts_sec

In [51]:
comfort_index = comfort_sec / total_sec

In [54]:
print("{:.0%}".format(comfort_index))

12%
